In [1]:
# !pip install torchvision
# !pip install torch

In [2]:
from PIL import Image, ImageEnhance, ImageOps
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from zipfile import ZipFile
import shutil

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class LeafDiseaseModel(nn.Module):
    def __init__(self):
        super(LeafDiseaseModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(64 * 32 * 32, 512) 
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        
        x = x.view(x.size(0), -1) 
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

data_transform = {'train': transforms.Compose([
        transforms.Resize((128, 128)),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomRotation(30),
        transforms.ToTensor(),
        ]),
            'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
        ])
    }
def load_data(data_dir):
    dataset = datasets.ImageFolder(data_dir, transform=data_transform['train'])
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset
def train_model(data_dir, output_dir):
    stop = 0.001
    prev_loss = 1
    train_dataset, val_dataset = load_data(data_dir)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=8, drop_last=True)
    prev_model = None

    model = LeafDiseaseModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(100):
        model.train()
        running_loss = 0.0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.float()
            batch_y = batch_y.long()
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                preds = model(batch_x)
                _, predicted = torch.max(preds, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()

            
        accuracy = correct / total
        print(f"Validation Accuracy: {accuracy * 100:.2f}%")
        if prev_loss - loss < stop or loss > prev_loss:
            break
        prev_model = model
        prev_loss = loss 

    model_path = os.path.join(output_dir, 'leaf_disease_model.pth')
    print(f"Saving model to {model_path}")
    try:
        torch.save(model.state_dict(), model_path)
    except Exception as e:
        print(f"Error saving model: {e}")
    zip_path = os.path.join(output_dir, 'results.zip')
    print(f"Creating zip file at {zip_path}")
    with ZipFile(zip_path, 'w') as zipf:
        if os.path.exists(model_path):
            zipf.write(model_path)
        else:
            print(f"File {model_path} does not exist, cannot add to zip")
        for subdir, dirs, files in os.walk(output_dir):
            for file in files:
                zipf.write(os.path.join(subdir, file))
    return prev_model
            
            
data_dir = 'images/Apple/'
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)
model_out = train_model(data_dir, output_dir)      

Epoch 1, Loss: 0.673989072477199
Validation Accuracy: 89.40%
Epoch 2, Loss: 0.3034481065810523
Validation Accuracy: 91.30%
Epoch 3, Loss: 0.22868600175218107
Validation Accuracy: 91.93%
Epoch 4, Loss: 0.1715313134489452
Validation Accuracy: 93.51%
Epoch 5, Loss: 0.13281560753349145
Validation Accuracy: 89.56%


In [6]:
import torch
from PIL import Image

def load_model(model_path, device='cpu'):
    model = LeafDiseaseModel()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Set the model to evaluation mode
    return model
    
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')  # Open the image and ensure it's in RGB mode
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension
    return image

def predict(image_path, model, transform, device='cpu'):
    image_tensor = preprocess_image(image_path, transform)
    image_tensor = image_tensor.to(device)

    with torch.no_grad():  # Disable gradient computation for inference
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()

# Path to your trained model
model_path = 'output/leaf_disease_model.pth'

# Define the same transforms as during training
infer_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

model_out.eval()

# Path to the input image for inference
image_path = 'images/Apple/apple_black_rot/image (100).JPG'

# Run the inference
predicted_class = predict(image_path, model_out, infer_transform, device)
print(f'Predicted class: {predicted_class}')

    

Predicted class: 0
